In [ ]:
import ollama
import gradio as gr
import json

ticket_prices = {"london": "$500", "paris": "$700", "tokyo": "$400"}

def get_ticket_price(destination_city):
    return ticket_prices.get(destination_city.lower(), "Unknown")

price_func={
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this function whenever you need to know the ticket price, for example when a customer asks 'How mcuh is a ticket price to this city'",
    "parameters":{
        "type": "object",
        "properties":{
            "destination_city":{
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_func}]


def chat(message, history):
    messages = [{"role": "system", "content": "You are a helpful assistant."}] + history + [{"role": "user", "content": message}]

    response = ollama.chat(model="llama3.2", messages=messages, tools=tools)

    if response['message'].get('tool_calls'):
        tool_response = handle_tool_call(response['message'])
        messages.append(response['message'])
        messages.append(tool_response)

        result = ollama.chat(model="llama3.2", messages = messages)
    return result.message.content

def handle_tool_call(message):
    tool_call = message['tool_calls'][0]
    arguments = tool_call['function']['arguments']
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response={
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        #"tool_call_id": 0
    }
    return response

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)  